# Chapter 17. 딥러닝을 이용한 자연어 처리





In [ ]:
import sys
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(f"python: {sys.version}")
print(f"numpy: {np.__version__}")
print(f"pandas: {pd.__version__}")
print(f"sklearn: {sklearn.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"seaborn: {sns.__version__}")
print(f"tensorflow: {tf.__version__}")

python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
numpy: 1.25.2
pandas: 1.5.3
sklearn: 1.2.2
matplotlib: 3.7.1
seaborn: 0.13.1
tensorflow: 2.15.0


In [ ]:
text = '해보지 않으면 해낼 수 없다'

result = text_to_word_sequence(text)
print("\n원문:\n", text)
print("\n토큰화:\n", result)


원문:
 해보지 않으면 해낼 수 없다

토큰화:
 ['해보지', '않으면', '해낼', '수', '없다']


In [ ]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다.',
       ]

token = Tokenizer()
token.fit_on_texts(docs)

print("\n단어 카운트:\n", token.word_counts)

print("\n문장 카운트: ", token.document_count)
print("\n각 단어가 몇 개의 문장에 포함되어 있는가:\n", token.word_docs)
print("\n각 단어에 매겨진 인덱스 값:\n",  token.word_index)


단어 카운트:
 OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])

문장 카운트:  3

각 단어가 몇 개의 문장에 포함되어 있는가:
 defaultdict(<class 'int'>, {'각': 1, '먼저': 1, '나누어': 1, '단어를': 1, '토큰화합니다': 1, '텍스트의': 2, '토큰화해야': 1, '딥러닝에서': 2, '인식됩니다': 1, '단어로': 1, '수': 1, '결과는': 1, '사용할': 1, '있습니다': 1, '토큰화한': 1})

각 단어에 매겨진 인덱스 값:
 {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화한': 11, '결과는': 12, '사용할': 13, '수': 14, '있습니다': 15}


In [ ]:
text="오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
token = Tokenizer()
token.fit_on_texts([text])
print(token.word_index)

{'오랫동안': 1, '꿈꾸는': 2, '이는': 3, '그': 4, '꿈을': 5, '닮아간다': 6}


In [ ]:
x=token.texts_to_sequences([text])
print(x)

[[1, 2, 3, 4, 5, 6]]


In [ ]:
word_size = len(token.word_index) + 1
x = to_categorical(x, num_classes=word_size)
print(x)

[[[0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]


원-핫 인코딩을 그대로 사용하면 벡터의 길이가 너무 길어진다는 단점

이러한 공간적 낭비를 해결하기 위해 등장한 것이 단어 임베딩(word embedding)

단어 임베딩으로 얻은 결과가 밀집된 정보를 가지고 있고 공간의 낭비가 적음 -> 이러한 결과가 가능한 이유는 각 단어 간의 유사도를 계산 했기 때문

이 단어 간 유사도는 어떻게 계산?

> 적절한 크기로 배열을 바꾸어 주기 위해 최적의 유사도를 계산 하는 학습 과정을 거치는 것 -> 오차 역전파 사용

In [ ]:
docs = [
    "너무 재밌네요",
    "최고예요",
    "참 잘 만든 영화예요",
    "추천하고 싶은 영화입니다",
    "한번 더 보고싶네요",
    "글쎄요",
    "별로예요",
    "생각보다 지루하네요",
    "연기가 어색해요",
    "재미없어요"
]

classes = np.array([1,1,1,1,1,0,0,0,0,0])

token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)


{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [ ]:
x = token.texts_to_sequences(docs)
print("\n리뷰 텍스트, 토큰화 결과:\n",  x)


리뷰 텍스트, 토큰화 결과:
 [[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


딥러닝 모델에 입력하려면 학습 데이터의 길이가 동일해야 함 -> 이처럼 길이를 똑같이 맞추어 주는 작업을 패딩(padding) 이라고 함

In [ ]:
padded_x = pad_sequences(x, 4)
print("\n패딩 결과:\n", padded_x)


패딩 결과:
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [ ]:
word_size = len(token.word_index) + 1

model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 8)              168       
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 201 (804.00 Byte)
Trainable params: 201 (804.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_x, classes, epochs=20)
print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6957 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6938 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6918 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6898 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 13ms/step - loss: 0.6879 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6859 - accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6840 - accuracy: 0.7000
Epoch 8/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6820 - accuracy: 0.7000
Epoch 9/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6801 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6781 - accuracy: 0.8000
Epoch 11/20
1/1 [=================